In [1]:
print('Hi, nice to meet you')

Hi, nice to meet you


In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
inputDF = glueContext.create_dynamic_frame_from_options(connection_type="s3",
                                                        connection_options = {"paths": ["s3://ti-dev-terraform-12345/raw/orders/orders.csv"]}, format = "csv")
inputDF.show(5)
df = inputDF.toDF()
df_pd = df.toPandas()
print('test script has successfully ran')

In [3]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
inputDF = glueContext.create_dynamic_frame_from_options(connection_type="s3",
                                                        connection_options = {"paths": ["s3://ti-dev-terraform-12345/raw/orders/orders.csv"]}, format = "csv")
inputDF.show(5)
df = inputDF.toDF()
df_pd = df.toPandas()
print('test script has successfully ran')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


{"col0": "salesorderid", "col1": "salesorderdetailid", "col2": "orderdate", "col3": "duedate", "col4": "shipdate", "col5": "employeeid", "col6": "customerid", "col7": "subtotal", "col8": "taxamt", "col9": "freight", "col10": "totaldue", "col11": "productid", "col12": "orderqty", "col13": "unitprice", "col14": "unitpricediscount", "col15": "linetotal"}
{"col0": "43659", "col1": "1", "col2": "5/31/2011", "col3": "6/12/2011", "col4": "6/7/2011", "col5": "279", "col6": "1045", "col7": "20565.6206", "col8": "1971.5149", "col9": "616.0984", "col10": "23153.2339", "col11": "776", "col12": "1", "col13": "2024.994", "col14": "0", "col15": "2024.994"}
{"col0": "43659", "col1": "2", "col2": "5/31/2011", "col3": "6/12/2011", "col4": "6/7/2011", "col5": "279", "col6": "1045", "col7": "20565.6206", "col8": "1971.5149", "col9": "616.0984", "col10": "23153.2339", "col11": "777", "col12": "3", "col13": "2024.994", "col14": "0", "col15": "6074.982"}
{"col0": "43659", "col1": "3", "col2": "5/31/2011", "c

/usr/local/lib/python3.11/site-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


test script has successfully ran


In [56]:
from pyspark.sql import SparkSession

# Create Spark session (Glue image already configures this)
spark = (
    SparkSession.builder
    .appName("ReadCustomersFromS3")
    .getOrCreate()
)

# S3 path
s3_path = "s3://ti-unique-bucket-12345/customers/"

# Read CSV from S3
df = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv(s3_path)
)

# Show data
df.show(truncate=False)
df.printSchema()




26/01/31 17:11:47 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+---+----------+-----------+--------------------+
|id |firstname |lastname   |fullname            |
+---+----------+-----------+--------------------+
|293|Catherine |Abel       |Catherine Abel      |
|295|Kim       |Abercrombie|Kim Abercrombie     |
|297|Humberto  |Acevedo    |Humberto Acevedo    |
|291|Gustavo   |Achong     |Gustavo Achong      |
|299|Pilar     |Ackerman   |Pilar Ackerman      |
|305|Carla     |Adams      |Carla Adams         |
|301|Frances   |Adams      |Frances Adams       |
|307|Jay       |Adams      |Jay Adams           |
|309|Ronald    |Adina      |Ronald Adina        |
|311|Samuel    |Agcaoili   |Samuel Agcaoili     |
|313|James     |Aguilar    |James Aguilar       |
|315|Robert    |Ahlering   |Robert Ahlering     |
|319|Kim       |Akers      |Kim Akers           |
|441|Stanley   |Alan       |Stanley Alan        |
|323|Amy       |Alberts    |Amy Alberts         |
|325|Anna      |Albright   |Anna Albright       |
|327|Milton    |Albury     |Milton Albury       |


In [61]:
from awsglue.dynamicframe import DynamicFrame

dyf = DynamicFrame.fromDF(
    df,
    glue_context,
    "customers_dyf"
)


In [68]:
database = 'dev_staging_test'

staging_path = "s3://ti-glue-catalog-bucket-12345/staging/"
project_folder = "customers/"
target_path = staging_path+project_folder

In [ ]:
from awsglue.context import GlueContext
from awsglue.dynamicframe import DynamicFrame
from pyspark.context import SparkContext

sc = SparkContext.getOrCreate()
glue_context = GlueContext(sc)

# Suppose df is your Spark DataFrame
dyf = DynamicFrame.fromDF(df, glue_context, "dyf")

glue_context.write_dynamic_frame.from_options(
    frame = dyf,
    connection_type = "s3",
    connection_options = {
        "path": "s3://ti-glue-catalog-bucket-12345/staging/customers/",
        "partitionKeys": []   # Add if you want partitions
    },
    format = "parquet",
    format_options = {"compression": "snappy"},
    transformation_ctx = "s3_write"
)


In [76]:
dyf.show()

{"id": 293, "firstname": "Catherine", "lastname": "Abel", "fullname": "Catherine Abel"}
{"id": 295, "firstname": "Kim", "lastname": "Abercrombie", "fullname": "Kim Abercrombie"}
{"id": 297, "firstname": "Humberto", "lastname": "Acevedo", "fullname": "Humberto Acevedo"}
{"id": 291, "firstname": "Gustavo", "lastname": "Achong", "fullname": "Gustavo Achong"}
{"id": 299, "firstname": "Pilar", "lastname": "Ackerman", "fullname": "Pilar Ackerman"}
{"id": 305, "firstname": "Carla", "lastname": "Adams", "fullname": "Carla Adams"}
{"id": 301, "firstname": "Frances", "lastname": "Adams", "fullname": "Frances Adams"}
{"id": 307, "firstname": "Jay", "lastname": "Adams", "fullname": "Jay Adams"}
{"id": 309, "firstname": "Ronald", "lastname": "Adina", "fullname": "Ronald Adina"}
{"id": 311, "firstname": "Samuel", "lastname": "Agcaoili", "fullname": "Samuel Agcaoili"}
{"id": 313, "firstname": "James", "lastname": "Aguilar", "fullname": "James Aguilar"}
{"id": 315, "firstname": "Robert", "lastname": "

In [95]:
spark.sql(F"DROP TABLE {database}.customers")

DataFrame[]

In [84]:
df.createOrReplaceTempView("tmpdf")

In [101]:
spark.sql(f"""
    CREATE TABLE {database}.customers
    USING parquet
    LOCATION 's3://ti-glue-catalog-bucket-12345/staging/customers/'
    AS
    SELECT *
    FROM tmpdf
""")


DataFrame[]